## LFR benchmark for Altmap vs Map Eq
### Compare altmap to map eq using networkx


In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict

plt.rcParams.update({'font.size': 20})
%pylab

%run helpers.py
# loads the following helper functions:
# infomap(net_path, altmap=False, additional_args='')
# read_tree(tree_path)
# plogq(p, q)
# plogp(p)
# drawNetwork(G, communities)
# altmap_cost(G, communities)
# create_initfile(G, N_partitions=None, randomized=True)
# generate_two_rings(n_ring=10)
# 


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [35]:
from networkx.algorithms.community.community_generators import LFR_benchmark_graph
from sklearn.metrics import normalized_mutual_info_score as nmi_score

# generate LFR benchmark graph + extract ground truth communities
def generate_LFR_benchmark(N = 250, mu = 0.1):
    
    # LFR params
    max_degree = int(0.2*N)
    max_community = int(0.2*N)
    min_community = 20
    average_degree = 10
    tau1 = 2.1 # Power law exponent for the degree distribution 
    tau2 = 1.1 # Power law exponent for the community size distribution

    # generate LFR benchmark graph
    G = LFR_benchmark_graph(N, tau1, tau2, mu, average_degree=average_degree, max_degree=max_degree, 
                            max_community=max_community, min_community=min_community)
    
    G = nx.convert_node_labels_to_integers(G, first_label=1)
    
    # extract ground truth communities from networkx graph object
    communities_true = {}
    num_communities = 0
    for n in range(1,N+1):
        if n in communities_true:
            continue
            
        num_communities = num_communities + 1
        community = G.nodes[n]['community']
        node_ids = np.asarray(list(community))
        node_ids = node_ids + 1 # have node labels >= 1
        communities_true.update(dict.fromkeys(node_ids , num_communities))
        
    communities_true = OrderedDict(sorted(communities_true.items()))
    
    return G, communities_true

# compute normalized mutual information between two partitions
def compute_nmi(communities_true, communities_found):
    labels_true = list(communities_true.values())
    labels_found = list(communities_found.values())

    return nmi_score(labels_true,labels_found, average_method='arithmetic')

# benchmark infomap
def run_benchmark(cost_function = 'altmap', init='std'):
    
    if cost_function not in {'altmap', 'mapeq'}:
        cost_function = 'altmap'
    altmap = (cost_function == 'altmap')
        
    if init not in {'std', 'random'}:
        init = 'std'
        
    N = 300
    mus = [0.03, 0.1, 0.3, 0.5] # [0.03, 0.75]
    for mu in mus:
        G, communities_true = generate_LFR_benchmark(N, mu)
        
        nx.write_pajek(G, workspace_path +  filename + '.net')
        
        if init == 'random':
            communities = create_initfile(G, randomized=True)
            infomap(workspace_path +  filename + '.net', altmap=altmap, additional_args=' --cluster-data ./workspace/init.tree')
        else:
            infomap(workspace_path +  filename + '.net', altmap=altmap)
            
        communities_found = read_communities_from_tree_file()
    
        num_communities_found = max(communities_found.values()) - min(communities_found.values()) + 1
        num_communities_true = max(communities_true.values()) - min(communities_true.values()) + 1
        print (f'We found {num_communities_found} communities vs. {num_communities_true} ground truth communities.\n')
        
        nmi = compute_nmi(communities_true, communities_found)
        print (f'Normalized mutual information I(.,.) = {nmi}.\n')


In [37]:
run_benchmark(cost_function = 'altmap', init='std')


ExceededMaxIterations: Could not assign communities; try increasing min_community

In [ ]:
plt.close('all')
plt.figure()
plt.title('Ground Truth Communities')
drawNetwork(G, communities_true, labels=False)

# 
# plt.figure()
# plt.title('Infomap/Altmap Communities')
# drawNetwork(G, communities_found, labels=False)
